In [0]:
import numpy as np
import math
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.optimizer import Optimizer

In [0]:
EPOCHS = 1000
EPOCHS_LOG_INTERVAL = 10

In [3]:
# import some data to play with
iris = datasets.load_iris()

print('Example data: ')
print(iris.data[:5])
print('Example labels: ')
print(iris.target[:5])

Example data: 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example labels: 
[0 0 0 0 0]


In [0]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X


x = iris.data
y = iris.target

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

train_X = Variable(torch.Tensor(train_x).float())
test_X = Variable(torch.Tensor(test_x).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())

In [62]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    out = model(train_X)
    loss = criterion(out, train_y)
    loss.backward()
    optimizer.step()
    
    if epoch % EPOCHS_LOG_INTERVAL == 0:
        print ('Epoch #', epoch, 'loss: ', loss.data.item())

predict_out = model(test_X)
_, predict_y = torch.max(predict_out, 1)
print ('prediction accuracy', accuracy_score(test_y.data, predict_y.data))

Epoch # 0 loss:  1.1231729984283447
Epoch # 10 loss:  1.0845417976379395
Epoch # 20 loss:  1.061902642250061
Epoch # 30 loss:  1.040486216545105
Epoch # 40 loss:  1.017595648765564
Epoch # 50 loss:  0.9945967793464661
Epoch # 60 loss:  0.9729045629501343
Epoch # 70 loss:  0.9532913565635681
Epoch # 80 loss:  0.9359519481658936
Epoch # 90 loss:  0.9207199811935425
Epoch # 100 loss:  0.9072737097740173
Epoch # 110 loss:  0.8952866196632385
Epoch # 120 loss:  0.8844863176345825
Epoch # 130 loss:  0.8746868371963501
Epoch # 140 loss:  0.865768551826477
Epoch # 150 loss:  0.8576305508613586
Epoch # 160 loss:  0.8501619696617126
Epoch # 170 loss:  0.8432596325874329
Epoch # 180 loss:  0.8368533849716187
Epoch # 190 loss:  0.8308594822883606
Epoch # 200 loss:  0.8252092003822327
Epoch # 210 loss:  0.8198577165603638
Epoch # 220 loss:  0.8147523999214172
Epoch # 230 loss:  0.8098625540733337
Epoch # 240 loss:  0.8051660656929016
Epoch # 250 loss:  0.8006303906440735
Epoch # 260 loss:  0.796232

In [56]:
import torch
from torch.optim import Optimizer
import torch.nn as nn
import numpy as np
import math

class SimulatedAnnealing(Optimizer):
    def __init__(self, params, t0 = 1.0
                 , anneal_rate = 0.0003, neighborhoodSize = 0.05
                 , loss = nn.CrossEntropyLoss()
                 , model = None
                 , features = None
                 , labels = None): #these represent default values, but can be overridden
        self.t0 = t0
        self.anneal_rate = anneal_rate
        self.t = t0
        self.loss = loss
        self.model = model
        self.features = features
        self.labels = labels
        self.neighborhoodSize = neighborhoodSize
        self.iterations = 1
        self.min_t = 1e-5

    def step(self):
        old_loss = self.loss(self.model(self.features), self.labels.type(torch.LongTensor))
        old_state_dict = {}
        for key in self.model.state_dict():
            old_state_dict[key] = self.model.state_dict()[key].clone()

        for name, param in self.model.state_dict().items():
            if (len(param.shape) == 2):
                random = torch.Tensor(np.random.uniform(low = self.neighborhoodSize * -1, high = self.neighborhoodSize
                                                        , size = (param.shape[0], param.shape[1])))
            else:
                random = torch.Tensor(np.random.uniform(low = self.neighborhoodSize * -1, high = self.neighborhoodSize
                                                        , size = param.shape[0]))
                                                        
            new_param = param + random
            self.model.state_dict()[name].copy_(new_param)
            self.t = max(self.t0 * self.anneal_rate / self.iterations, self.min_t)
            self.iterations += 1

        newPerformance = self.loss(self.model(self.features), self.labels.type(torch.LongTensor))
        if (newPerformance > old_loss):
            jumpProb = torch.exp((newPerformance - old_loss) / self.t) 
            if (np.random.uniform(0, 1) > jumpProb): 
                self.model.load_state_dict(old_state_dict)


model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = SimulatedAnnealing(
    params = model.parameters(), model = model, features = train_X, labels = train_y
)

for epoch in range(EPOCHS):
  output = model(train_X)
  loss = criterion(output, train_y)
  optimizer.step()
  if epoch % 100 == 0:
    print ('number of epoch', epoch, ' loss ', loss.data.item())

predict_out = model(test_X)
_, predict_y = torch.max(predict_out, 1)
print ('prediction accuracy', accuracy_score(test_y.data, predict_y.data))

  

number of epoch 0  loss  1.1110694408416748
number of epoch 100  loss  1.2180982828140259
number of epoch 200  loss  1.2347773313522339
number of epoch 300  loss  1.2070424556732178
number of epoch 400  loss  1.2181106805801392
number of epoch 500  loss  1.2181106805801392
number of epoch 600  loss  1.2347760200500488
number of epoch 700  loss  1.2401587963104248
number of epoch 800  loss  1.2181106805801392
number of epoch 900  loss  1.2014440298080444
number of epoch 1000  loss  1.2014440298080444
number of epoch 1100  loss  1.2014440298080444
number of epoch 1200  loss  1.2014440298080444
number of epoch 1300  loss  1.2014440298080444
number of epoch 1400  loss  1.2014440298080444
number of epoch 1500  loss  1.2014440298080444
number of epoch 1600  loss  1.2014440298080444
number of epoch 1700  loss  1.2014440298080444
number of epoch 1800  loss  1.2014440298080444
number of epoch 1900  loss  1.2014440298080444
number of epoch 2000  loss  1.2014440298080444
number of epoch 2100  los